# Shopify Capstone: Going from raw data in bz2 format to clean data

In [1]:
# import all necessary modules and submodules
import bz2
import datetime
import sys
import numpy
import json
import re
from datetime import timezone
import pandas

In [2]:
# define filenames ranges
periods = ['2015-11','2015-12','2016-01','2016-02','2016-03','2016-04',
           '2016-05','2016-06','2016-07','2016-08','2016-09','2016-10']
namesRS = []
namesRC = []
for period in periods:
    RS_bz2 = 'data/raw/RS_'+period+'.bz2'
    RS_json = 'data/RS_'+period+'.json'
    RC_bz2 = 'data/raw/RC_'+period+'.bz2'
    RC_json = 'data/RC_'+period+'.json'
    namesRS.append([RS_json,RS_bz2])
    namesRC.append([RC_json,RC_bz2])
fileNames = namesRC + namesRS
filesAll = numpy.array(fileNames)
filesRC = filesAll[0:12,0]
filesRS = filesAll[12:24,0]
filesJSON = filesAll[:,0]

In [19]:
# removing links from RC files
for file in filesRC:
    print(file)
    with open(file.replace(".json", "-no_links.json"),'w',errors='surrogateescape') as output:
        with open(file,'rt', errors='surrogateescape') as input_:
            for line in input_:
                line_json = json.loads(line)
                body = line_json["body"]
                line_json["body"] = re.sub('\(?(https?://|www.)[^ ,\n"]*',"---LINK---", body, flags=re.I)
                new_line = json.dumps(line_json)+"\n"
                if 'shopify' in new_line.lower():
                    output.write(new_line)

data/RC_2015-11.json
data/RC_2015-12.json
data/RC_2016-01.json
data/RC_2016-02.json
data/RC_2016-03.json
data/RC_2016-04.json
data/RC_2016-05.json
data/RC_2016-06.json
data/RC_2016-07.json
data/RC_2016-08.json
data/RC_2016-09.json
data/RC_2016-10.json


In [20]:
# keep only links with "shopify" in author, author_flair_text, title, selftext, or subreddit
exceptions = []
for file in filesRS:
    with open(file.replace(".json", "-four_fields.json"),'w',errors='surrogateescape') as output:
        with open(file,'rt', errors='surrogateescape') as input_:
            print(file)
            lines = input_.readlines()
            for line in lines:
                json_line = json.loads(line)
                xstr = lambda obj: "None" if obj is None else str(obj) + "  "
                fields = ["author", "selftext", "title", "subreddit", "author_flair_text"]
                subline = ""
                for field in fields:
                    try:
                        subline = subline + field + ": " + xstr(json_line[field])
                    except Exception as e:
                        exceptions.append(str(e))
                if "shopify" in subline.lower():
                    output.write(line)
print("exceptions:", set(exceptions))

data/RS_2015-11.json
data/RS_2015-12.json
data/RS_2016-01.json
data/RS_2016-02.json
data/RS_2016-03.json
data/RS_2016-04.json
data/RS_2016-05.json
data/RS_2016-06.json
data/RS_2016-07.json
data/RS_2016-08.json
data/RS_2016-09.json
data/RS_2016-10.json
exceptions: {"'subreddit'"}


In [3]:
# defining a list of names for cleanER RS files
filesFourFields = []
for file in filesRS:
    filesFourFields.append(file.replace(".json", "-four_fields.json"))
filesFourFields

['data/RS_2015-11-four_fields.json',
 'data/RS_2015-12-four_fields.json',
 'data/RS_2016-01-four_fields.json',
 'data/RS_2016-02-four_fields.json',
 'data/RS_2016-03-four_fields.json',
 'data/RS_2016-04-four_fields.json',
 'data/RS_2016-05-four_fields.json',
 'data/RS_2016-06-four_fields.json',
 'data/RS_2016-07-four_fields.json',
 'data/RS_2016-08-four_fields.json',
 'data/RS_2016-09-four_fields.json',
 'data/RS_2016-10-four_fields.json']

In [22]:
# removing links from RS files
for file in filesFourFields:
    with open(file.replace("-four_fields.json", "-no_links.json"),'w',errors='surrogateescape') as output:
        with open(file,'rt', errors='surrogateescape') as input_:
            print(file)
            for line in input_:
                line_json = json.loads(line)
                
                title = line_json["title"]
                line_json["title"] = re.sub('\(?(https?://|www.)[^ ,\n"]*',"---LINK---", title, flags=re.I)
                
                selftext = line_json["selftext"]
                line_json["selftext"] = re.sub('\(?(https?://|www.)[^ ,\n"]*',"---LINK---", selftext, flags=re.I)
                
                author_flair_text = str(line_json["author_flair_text"])
                line_json["author_flair_text"] = re.sub('\(?(https?://|www.)[^ ,\n"]*',"---LINK---", author_flair_text, flags=re.I)
                
                try:
                    subreddit = line_json["subreddit"]
                except Exception as e:
                    subreddit = ""
                
                subline = line_json["author_flair_text"] + line_json["selftext"] + line_json["title"] + line_json["author"] + subreddit
                if "shopify" in subline.lower():
                    new_line = json.dumps(line_json)+"\n"
                    output.write(new_line)

data/RS_2015-11-four_fields.json
data/RS_2015-12-four_fields.json
data/RS_2016-01-four_fields.json
data/RS_2016-02-four_fields.json
data/RS_2016-03-four_fields.json
data/RS_2016-04-four_fields.json
data/RS_2016-05-four_fields.json
data/RS_2016-06-four_fields.json
data/RS_2016-07-four_fields.json
data/RS_2016-08-four_fields.json
data/RS_2016-09-four_fields.json
data/RS_2016-10-four_fields.json


In [4]:
# defining a list of names for clean files
filesNoLink = []
for file in filesJSON:
    filesNoLink.append(file.replace(".json", "-no_links.json"))
filesNoLink

['data/RC_2015-11-no_links.json',
 'data/RC_2015-12-no_links.json',
 'data/RC_2016-01-no_links.json',
 'data/RC_2016-02-no_links.json',
 'data/RC_2016-03-no_links.json',
 'data/RC_2016-04-no_links.json',
 'data/RC_2016-05-no_links.json',
 'data/RC_2016-06-no_links.json',
 'data/RC_2016-07-no_links.json',
 'data/RC_2016-08-no_links.json',
 'data/RC_2016-09-no_links.json',
 'data/RC_2016-10-no_links.json',
 'data/RS_2015-11-no_links.json',
 'data/RS_2015-12-no_links.json',
 'data/RS_2016-01-no_links.json',
 'data/RS_2016-02-no_links.json',
 'data/RS_2016-03-no_links.json',
 'data/RS_2016-04-no_links.json',
 'data/RS_2016-05-no_links.json',
 'data/RS_2016-06-no_links.json',
 'data/RS_2016-07-no_links.json',
 'data/RS_2016-08-no_links.json',
 'data/RS_2016-09-no_links.json',
 'data/RS_2016-10-no_links.json']

In [21]:
# find the set of links
links = []
for file in filesNoLink[:12]:
    print(file)
    with open(file,'rt', errors='surrogateescape') as input_:
        for line in input_:
            links.append(json.loads(line)['link_id'])
for file in filesNoLink[12:]:
    print(file)
    with open(file,'rt', errors='surrogateescape') as input_:
        for line in input_:
            links.append(json.loads(line)['name'])
links = sorted(set(links))
print("unique links:", len(links))

data/RC_2015-11-no_links.json
data/RC_2015-12-no_links.json
data/RC_2016-01-no_links.json
data/RC_2016-02-no_links.json
data/RC_2016-03-no_links.json
data/RC_2016-04-no_links.json
data/RC_2016-05-no_links.json
data/RC_2016-06-no_links.json
data/RC_2016-07-no_links.json
data/RC_2016-08-no_links.json
data/RC_2016-09-no_links.json
data/RC_2016-10-no_links.json
data/RS_2015-11-no_links.json
data/RS_2015-12-no_links.json
data/RS_2016-01-no_links.json
data/RS_2016-02-no_links.json
data/RS_2016-03-no_links.json
data/RS_2016-04-no_links.json
data/RS_2016-05-no_links.json
data/RS_2016-06-no_links.json
data/RS_2016-07-no_links.json
data/RS_2016-08-no_links.json
data/RS_2016-09-no_links.json
data/RS_2016-10-no_links.json
unique links: 10350


In [6]:
# find first links of each month (Reddit)
first_links = []
for file in filesAll[12:24,1]:
    print(file)
    progress = 0
    with bz2.open(file,'rt') as input_:
        for line in input_:
            progress = progress + 1
            if progress > 1:
                break
            line_json = json.loads(line)
            name = line_json["name"]
            first_links.append(name)
first_links

data/raw/RS_2015-11.bz2
data/raw/RS_2015-12.bz2
data/raw/RS_2016-01.bz2
data/raw/RS_2016-02.bz2
data/raw/RS_2016-03.bz2
data/raw/RS_2016-04.bz2
data/raw/RS_2016-05.bz2
data/raw/RS_2016-06.bz2
data/raw/RS_2016-07.bz2
data/raw/RS_2016-08.bz2
data/raw/RS_2016-09.bz2
data/raw/RS_2016-10.bz2


['t3_3r0zf4',
 't3_3uxbvg',
 't3_3yyun4',
 't3_43lprb',
 't3_48ddvx',
 't3_4cstvr',
 't3_4h6v0n',
 't3_4lyd4m',
 't3_4qotdt',
 't3_4vjrss',
 't3_50kc60',
 't3_55atoy']

In [7]:
integrator = lambda string: int(string.replace("t3_",""),36)

In [8]:
# define borders as the first link of each month
borders = []
for link in first_links:
    borders.append(integrator(link))
borders

[226794064,
 233342044,
 240131488,
 247916711,
 255926157,
 263365191,
 270738311,
 278739958,
 286692545,
 294855292,
 303279768,
 311234002]

In [9]:
# find indices of first links in each month (our database)
links_base10 = []
for link in links:
    links_base10.append(integrator(link))
last_for_month = []
j = 0
for i in range(0,len(links_base10)-1): 
    if links_base10[i] > borders[j+1]:
        last_for_month.append(i)
        if j < len(borders)-1-1:             #another -1 so that we only run through
            j = j + 1
        else:
            break
last_for_month.append(len(links_base10))    
last_for_month

[817, 1379, 2074, 2861, 3682, 4479, 5313, 6181, 7150, 8190, 9241, 10350]

In [11]:
# compile the list of search strings by month
search_strings = []
for index in last_for_month:
    search_strings.append(re.compile("|".join(links[:index])))
search_strings[0]

re.compile(r't3_374967|t3_38ye7o|t3_3bgbra|t3_3bq1ii|t3_3bqwm1|t3_3d2syk|t3_3d68w0|t3_3fq1ny|t3_3gi1j6|t3_3goep1|t3_3gzu63|t3_3h49zn|t3_3h5lko|t3_3hgy3u|t3_3iamiw|t3_3id72c|t3_3idgjj|t3_3isfro|t3_3iw2wi|t3_3jbpg8|t3_3jh533|t3_3kth8c|t3_3l6o8j|t3_3lgqyv|t3_3m3l5b|t3_3muv5c|t3_3mxg5d|t3_3n8r3z|t3_3n91a3|t3_3nmdcw|t3_3npd5e|t3_3nq36h|t3_3nvmtp|t3_3o4vn7|t3_3ofarn|t3_3p2cft|t3_3p2ll8|t3_3p9b0v|t3_3pezp1|t3_3pixah|t3_3pj9d5|t3_3pni20|t3_3pnthc|t3_3ppv1z|t3_3pu3l5|t3_3pxdc0|t3_3pxp5f|t3_3q6uwu|t3_3q6w1y|t3_3q75t4|t3_3q8kbu|t3_3qa9ld|t3_3qbtqz|t3_3qc3be|t3_3qcc87|t3_3qcp2y|t3_3qfbkl|t3_3qgplm|t3_3qj7tn|t3_3qkl49|t3_3qme29|t3_3qobx0|t3_3qpeau|t3_3qq4n7|t3_3qrwlq|t3_3qv1jx|t3_3qv3al|t3_3qwc20|t3_3qx74u|t3_3qx7jo|t3_3qxxwv|t3_3r0jxa|t3_3r13ry|t3_3r1sz9|t3_3r27x8|t3_3r2uhg|t3_3r2xjv|t3_3r3ngh|t3_3r3z3w|t3_3r4ciw|t3_3r4fav|t3_3r4wkl|t3_3r4xs1|t3_3r525t|t3_3r6isv|t3_3r6n5w|t3_3r73g7|t3_3r7ffq|t3_3r7fpy|t3_3r7fu0|t3_3r7t9g|t3_3r7u37|t3_3r83my|t3_3r8bhn|t3_3r8ctf|t3_3r8hzo|t3_3r8vfp|t3_3r9ctf|t3_3r9h

In [22]:
search_strings_RS = []
first = 0
links_RS = []
for link in links:
    links_RS.append(link.replace("t3_",""))
for last in last_for_month: 
    search_strings_RS.append(re.compile("|".join(links_RS[first:last])))
    first = last
search_strings_RS[11]

re.compile(r'55awms|55b45b|55bo2i|55bshi|55c2o6|55c3ek|55c42e|55cen2|55cfri|55chh4|55ct9c|55cthm|55cwvv|55d97o|55df95|55dxvx|55dyqr|55e35p|55e4wo|55eeio|55eimr|55f4a7|55f73x|55fcic|55fjw1|55fl1c|55fpmq|55frsv|55fx2r|55g47w|55h07o|55h789|55h7jw|55hc8a|55hhy2|55hp3n|55iatg|55irx9|55iu6m|55j1wg|55j3s1|55j4ak|55j6o1|55j8b7|55ja3f|55jbtb|55jetg|55jgp2|55jn33|55joxb|55jsxz|55jud1|55k1tu|55koqb|55l1np|55l708|55lg1s|55m0ij|55mfdq|55mgzr|55mm72|55mnzm|55mp2l|55mrvn|55mx9u|55n20x|55n2ny|55n32y|55n3y4|55n4o1|55n65p|55nape|55nf7e|55njm5|55nlf0|55nmo8|55noqr|55npof|55nru2|55nt8i|55nulh|55nv5h|55nw4p|55ok9l|55ov3q|55oyvk|55oz17|55p8py|55pnwr|55ppov|55q996|55qich|55qps9|55qu9s|55qxre|55qykh|55qzwg|55r0qj|55r98z|55rjrm|55rkq4|55rl4d|55rmv2|55rsym|55rvrr|55rwn9|55rzpg|55shu9|55sjcw|55sx25|55t06m|55t2n9|55t4r2|55t6bl|55t9j9|55titw|55tjsj|55tmya|55tpu5|55ts0u|55tu3a|55tx9h|55tyz3|55tz65|55ulwj|55unxj|55urkw|55uv05|55uw9q|55v2g2|55v8ob|55vjpk|55vv4o|55w24g|55wao7|55wo6s|55xeq1|55xv3r|55y79u|55y7jm|55y8sf|

In [23]:
# generate files with all comments from the relevant links
i = 0  ### change this to zero !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
for file in filesAll[12:24,1]:
    
    new_file = file.replace(".bz2", "-db.json")
    new_file = new_file.replace("data/raw/", "data/")
    
    print(new_file)
    
    with open(new_file,'w') as w:

        #progress = 0
        start = datetime.datetime.now()

        with bz2.open(file, 'rt') as input_:
            for line in input_:
                progress = progress + 1
                link = json.loads(line)["id"]
                result = search_strings_RS[i].search(link)
                if result != None:
                    w.write(line)    
    
    i = i + 1
    print(datetime.datetime.now()-start)

data/RS_2015-11-db.json
0:12:19.833454
data/RS_2015-12-db.json
0:12:17.958864
data/RS_2016-01-db.json
0:13:40.104239
data/RS_2016-02-db.json
0:13:24.254560
data/RS_2016-03-db.json
0:16:13.458655
data/RS_2016-04-db.json
0:15:26.026585
data/RS_2016-05-db.json
0:13:29.893834
data/RS_2016-06-db.json
0:13:51.989681
data/RS_2016-07-db.json
0:14:12.813179
data/RS_2016-08-db.json
0:15:47.135877
data/RS_2016-09-db.json
0:14:53.718449
data/RS_2016-10-db.json
0:17:20.069631


In [13]:
# generate files with all comments from the relevant links
i = 5  ### change this to zero !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
for file in filesAll[5:12,1]:
    
    new_file = file.replace(".bz2", "-db.json")
    new_file = new_file.replace("data/raw/", "data/")
    
    print(new_file)
    
    with open(new_file,'w') as w:

        #progress = 0
        start = datetime.datetime.now()

        with bz2.open(file, 'rt') as input_:
            for line in input_:
                progress = progress + 1
                link = json.loads(line)["link_id"]
                result = search_strings[i].search(link)
                if result != None:
                    w.write(line)    
    
    i = i + 1
    print(datetime.datetime.now()-start)

data/RC_2016-04-db.json
1:33:38.166127
data/RC_2016-05-db.json
1:44:15.506261
data/RC_2016-06-db.json
1:56:02.898465
data/RC_2016-07-db.json
2:20:50.295060
data/RC_2016-08-db.json
4:50:52.096669
data/RC_2016-09-db.json
1:52:59.216263
data/RC_2016-10-db.json
2:31:51.953835


In [14]:
with open('data/RC_2016-01-db.json','w') as w:

    start = datetime.datetime.now()

    with bz2.open('data/raw/RC_2016-01.bz2', 'rt') as input_:
        for line in input_:
            link = json.loads(line)["link_id"]
            result = search_strings[2].search(link)
            if result != None:
                w.write(line)    
    
print(datetime.datetime.now()-start)

0:56:17.691615


# search all files for occurrences of the word "shopify"
for (file_in,file_out) in filesAll:
    start = datetime.datetime.now()
    print(file_in)
    with open(file_out,'w', errors='surrogateescape') as output:
        #progress = 0
        with bz2.open(file_in, 'rt', errors='surrogateescape') as input_:
            for line in input_:
                #progress = progress + 1
                #if progress > 100:
                #    break
                #if progress % 500000 == 0:
                #    print(progress)
                if 'shopify' in line.lower():
                    output.write(line)
        print(datetime.datetime.now()-start)

## 

In [2]:
def read_json_as_pandas(file_in):
    with open(file_in, 'r') as f:
        data = f.readlines()
        data = map(lambda x: x.rstrip(), data)
        data_json_str = "[" + ','.join(data) + "]"
        dataDf = pandas.read_json(data_json_str)
    return dataDf

In [3]:
file_in_04 = '/resources/data/Shopify/RC_2015-04.json'
file_in_05 = '/resources/data/Shopify/RC_2015-05.json'
file_in_06 = '/resources/data/Shopify/RC_2015-06.json'

In [4]:
shopifyDataDf04 = read_json_as_pandas(file_in_04)
shopifyDataDf04.shape

(1424, 22)

In [7]:
shopifyDataDf = (shopifyDataDf04.append(shopifyDataDf05,ignore_index=True)).append(shopifyDataDf06,ignore_index=True)
shopifyDataDf.shape

(3870, 22)

In [8]:
file_in_all_04 = '/resources/data/Shopify/RC_links_2015-04.json'
file_in_all_05 = '/resources/data/Shopify/RC_links_2015-05.json'
file_in_all_06 = '/resources/data/Shopify/RC_links_2015-06.json'

In [9]:
allDataDf04 = read_json_as_pandas(file_in_all_04)
allDataDf04.shape

(23818, 22)

In [10]:
allDataDf05 = read_json_as_pandas(file_in_all_05)
allDataDf05.shape

(12864, 22)

In [11]:
allDataDf06 = read_json_as_pandas(file_in_all_06)
allDataDf06.shape

(30363, 22)

In [12]:
allDataDf = (allDataDf04.append(allDataDf05,ignore_index=True)).append(allDataDf06,ignore_index=True)
allDataDf.shape

(67045, 22)

In [13]:
file_links_04 = '/resources/data/Shopify/RS_links_2015-04.json'
file_links_05 = '/resources/data/Shopify/RS_links_2015-05.json'
file_links_06 = '/resources/data/Shopify/RS_links_2015-06.json'

In [14]:
linkDf04 = read_json_as_pandas(file_links_04)
linkDf04.shape

(290, 41)

In [15]:
linkDf05 = read_json_as_pandas(file_links_05)
linkDf05.shape

(283, 41)

In [16]:
linkDf06 = read_json_as_pandas(file_links_06)
linkDf06.shape

(385, 50)

In [17]:
linkDf = (linkDf04.append(linkDf05,ignore_index=True)).append(linkDf06,ignore_index=True)
linkDf.shape

(958, 50)

In [18]:
URL_REGEX = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
SHOPIFY_REGEX = r"[S|s]hopify"
HIRING_REGEX = r"[H|h]iring"
HISTORY_REGEX = r"[R|r]ecent [S|s]ubmission [H|h]istory "

In [19]:
#list to store ids of comments to keep 
commentKeep =[]
for cid,comment in enumerate(shopifyDataDf['body']):
    if (len(re.findall(URL_REGEX,comment))>0):
        shopifyDataDf['body'][cid] = re.sub(URL_REGEX,'REMOVEDURL',comment)
    #only keep comments with links if they mention shopify w/o it occuring in link    
    if (len(re.findall(SHOPIFY_REGEX,shopifyDataDf['body'][cid]))>0):
            #remove any post with 'Hiring' or 'Recent Submission History'
            if(len(re.findall(HIRING_REGEX,comment))==0 and  len(re.findall(HISTORY_REGEX,comment))==0):
                commentKeep.append(cid)
print(len(commentKeep))

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1721


In [20]:
cleanDataDf = shopifyDataDf.loc[commentKeep]
cleanDataDf = cleanDataDf.reset_index(drop=True)

##### Step 5
Step 5 includes retrieving hierarchy for those comments which passed filtering.

links = list(cleanDataDf['link_id'].unique())
pickle.dump(obj=links,file=open('/resources/data/Shopify/links_2015-04,05.txt','wb'),protocol=2)
len(links)

import bz2
import datetime
import sys
import json
import pickle
import re

file_data = sys.argv[1]
file_links = sys.argv[2]
file_out = sys.argv[3]

links = pickle.load(open(file_links,'rb'))
search_string = re.compile("|".join(links))

w = open(file_out,'w')

progress = 0
start = datetime.datetime.now()

with bz2.BZ2File(file_data, 'r') as inputFile:
    for line in inputFile:
        progress = progress + 1
        if progress % 10000 == 0:
            print(progress)
        result = search_string.search(line)
        if result != None:
            w.write(line)    
w.close()
print(datetime.datetime.now()-start)

##### Step 6
Outputting clean data to a JSON file.

In [24]:
def find_ancestors_for_all(shopifyDf,allDf):
    parentsDf = shopifyDf
    parentsDf['parents']=None
    for i, name in parentsDf.ix[:,'name':'name'].itertuples():
        parentsDf['parents'][i]=find_ancestors_for_one(name,allDf)
        print(parentsDf[parentsDf['name']==name]['parents'].values)
    return parentsDf

In [25]:
def find_ancestors_for_one(name,allDf):
    print(name)
    try:
        parent = allDf[allDf['name']==name]['parent_id'].values[0]
    except:
        return ""
    if parent[:2] == "t1" and parent in allDf['name'].values:
        return find_ancestors_for_one(parent,allDf) + '/' + parent
    else:
        return parent

In [26]:
def add_thread_for_all(shopifyDf,allDf,linkDf):
    dataDf = shopifyDf
    dataDf['full_body']=dataDf['body']
    for i, row in dataDf.iterrows():
        add_thread_for_one(i,dataDf,allDf,linkDf)
    return dataDf

In [27]:
def add_thread_for_one(index,shopifyDf,allDf,linkDf):
    ancestors = parentDf['parents'][index].split("/")
    for ancestor in ancestors:
        if ancestor[:2] == "t3":
            try:
                shopifyDf['full_body'][index] = linkDf[linkDf['name']==ancestor]['title'].values[0] + "\n---------\n " + linkDf[linkDf['name']==ancestor]['selftext'].values[0] + "\n--- " + shopifyDf['full_body'][index]
            except:
                print('no link for', ancestor)
                continue
        else:
            try:
                shopifyDf['full_body'][index] = allDf[allDf['name']==ancestor]['body'].values[0] + "\n--- " + shopifyDf['full_body'][index]
            except:
                print('no comment for', ancestor)
                continue

In [28]:
parentDf = find_ancestors_for_all(cleanDataDf,allDataDf)

t1_cpxb1ih
['t3_30zhum']
t1_cpxbig4
t1_cpxapo0


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['t1_cpx8xrj/t1_cpxapo0']
t1_cpxcitr
['t3_30vv6d']
t1_cpxclc7
['t1_cpx66qr']
t1_cpxddwq
['t1_cpx5crl']
t1_cpxfp26
['t3_30zz0e']
t1_cpxfptx
['t3_30vu6b']
t1_cpxgh7b
t1_cpxg45l
t1_cpxg0ww
t1_cpxfy6m
t1_cpxfptx
['t3_30vu6b/t1_cpxfptx/t1_cpxfy6m/t1_cpxg0ww/t1_cpxg45l']
t1_cpxl9ze
t1_cpxfp26
['t3_30zz0e/t1_cpxfp26']
t1_cpxotu3
['t3_311gsl']
t1_cpxtdj6
t1_cpxhic7
['t3_30xh3y/t1_cpxhic7']
t1_cpxvwug
['t3_312ch2']
t1_cpxxbyh
['t3_312ch2']
t1_cpy0zb7
t1_cpxzj68
t1_cpxz90f
['t3_312vki/t1_cpxz90f/t1_cpxzj68']
t1_cpy1mrm
t1_cpxzecz
t1_cpxv8mz
t1_cpxpxxi
t1_cpxizjd
['t3_30zsjx/t1_cpxizjd/t1_cpxpxxi/t1_cpxv8mz/t1_cpxzecz']
t1_cpyarmo
t1_cpy54qj
t1_cpy50hn
['t3_31269n/t1_cpy50hn/t1_cpy54qj']
t1_cpyd2vz
t1_cpxjotu
['t1_cpudsre/t1_cpxjotu']
t1_cpymst0
['t3_315unf']
t1_cpymu60
['t3_312wol']
t1_cpymvcp
t1_cpymst0
['t3_315unf/t1_cpymst0']
t1_cpytna3
['t3_316ocs']
t1_cpyu6vi
t1_cpytv0o
['t3_315xvi/t1_cpytv0o']
t1_cpyud74
['t3_315xvi']
t1_cpyue5y
['t3_315unf']
t1_cpyue8k
t1_cpytrwr
t1_cpytna3
['t3_316ocs/t1

In [29]:
threadDf = add_thread_for_all(parentDf,allDataDf,linkDf)

no link for t3_30zhum
no comment for t1_cpx8xrj
no link for t3_30vv6d
no comment for t1_cpx66qr
no comment for t1_cpx5crl
no link for t3_30vu6b
no link for t3_30vu6b


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


no link for t3_30xh3y
no comment for t1_cpudsre
no link for t3_2vkse2
no comment for t1_cpw7055
no link for t3_30wsyx
no link for t3_2xazv5
no link for t3_2xazv5
no comment for t1_cpr47dl
no comment for t1_cpprwvy
no comment for t1_cpw7055
no link for t3_2mg0b7
no link for t3_30omym
no comment for t1_cpk4hkn
no comment for t1_cpjy7k6
no comment for t1_cpotblw
no comment for t1_cpl09rc
no comment for t1_co4gmdc
no link for t3_2rvrtp
no link for t3_30mo90
no link for t3_2rqu4o
no link for t3_2ompmp
no link for t3_2rg5km
no link for t3_2nvchb
no link for t3_35nb4x
no link for t3_2p5b9f
no link for t3_2w5upe
no link for t3_2wc8vj
no link for t3_2w31rj
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comment for 
no comme

In [32]:
finalDf = threadDf[:1084]

In [43]:
finalShopifySubDf = finalDf[finalDf['subreddit']=='shopify']

In [45]:
finalShopifySubDf.to_json('/resources/data/Shopify/shopify_Dec31.json',orient='records')

In [48]:
subs = ['ecommerce','smallbusiness','webdev','business','web_design','startups','Entrepreneurship','webdevelopment']
finalOtherSubsDf = finalDf[finalDf['subreddit'] in subs]

KeyError: False

In [39]:
finalDf.to_json('/resources/data/Shopify/Dec31.json',orient='records')

In [ ]:
pickle.dump(obj=finalDf,file=open('/resources/data/Shopify/fina.txt','wb'),protocol=2)

In [152]:
for cid,full_body in enumerate(finalDf['full_body']):
    if (len(re.findall(URL_REGEX,full_body))>0):
        finalDf['full_body'][cid] = re.sub(URL_REGEX,'REMOVEDURL',comment)

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.4/dist-packages/IPython/core/interactiveshell.py:2869: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


    parents = []
    name = comment_name
    while(True):
        try:
            parent_name = comments[name]["parent_id"]
        except:
            print("Comment: ", comment_name)
            print("\tOne of parent comments not in the database:", parent_name)
            print("\tTerminating execution")
            break
        parents.insert(0, parent_name)
        if parents[0][:2] == "t3":
            break
        name = parent_name
    return parents

def full_body(comment_name, parents, comments, submissions):
    full_text = comments[comment_name]['body']
    for parent in parents[::-1]:
        if parent[:2] == "t3":
            try:
                full_text = submissions[parent]["title"] + "\n---\n" + full_text
            except:
                print("Comment:", comment_name)
                print("\tSubmission", parent, "not in the database!")
                print("\tTerminating execution")
        else:
            try:
                full_text = comments[parent]["body"] + "\n---\n" + full_text
            except:
                print("Comment:", comment_name)
                print("\tParent comment", parent, "not in the database!")
                print("\tTerminating execution") 
    return full_text

#modified from http://www.quesucede.com/page/show/id/python-3-tree-implementation

(_ROOT, _DEPTH, _BREADTH) = range(3)

class Node:
    def __init__(self, identifier):
        self.__identifier = identifier
        self.__children = []

    @property
    def identifier(self):
        return self.__identifier

    @property
    def children(self):
        return self.__children

    def add_child(self, identifier):
        self.__children.append(identifier)


class Tree:

    def __init__(self):
        self.__nodes = {}

    @property
    def nodes(self):
        return self.__nodes

    def add_node(self, identifier, parent=None):
        node = Node(identifier)
        self[identifier] = node

        if parent is not None:
            self[parent].add_child(identifier)

        return node

    def display(self, identifier, depth=_ROOT):
        children = self[identifier].children
        if depth == _ROOT:
            print("{0}".format(identifier))
        else:
            print("\t"*depth, "{0}".format(identifier))

        depth += 1
        for child in children:
            self.display(child, depth)  # recursive call

    def traverse(self, identifier, mode=_DEPTH):
        # Python generator. Loosly based on an algorithm from 
        # 'Essential LISP' by John R. Anderson, Albert T. Corbett, 
        # and Brian J. Reiser, page 239-241
        yield identifier
        queue = self[identifier].children
        while queue:
            yield queue[0]
            expansion = self[queue[0]].children
            if mode == _DEPTH:
                queue = expansion + queue[1:]  # depth-first
            elif mode == _BREADTH:
                queue = queue[1:] + expansion  # width-first

    def __getitem__(self, key):
        return self.__nodes[key]

    def __setitem__(self, key, item):
        self.__nodes[key] = item

tree = Tree()

tree.add_node("Harry")  # root node
tree.add_node("Jane", "Harry")
tree.add_node("Bill", "Harry")
tree.add_node("Joe", "Jane")
tree.add_node("Diane", "Jane")
tree.add_node("George", "Diane")
tree.add_node("Mary", "Diane")
tree.add_node("Jill", "George")
tree.add_node("Carol", "Jill")
tree.add_node("Grace", "Bill")
tree.add_node("Mark", "Jane")

tree.display("Harry")
print("***** DEPTH-FIRST ITERATION *****")
for node in tree.traverse("Harry"):
    print(node)
print("***** BREADTH-FIRST ITERATION *****")
for node in tree.traverse("Harry", mode=_BREADTH):
    print(node)

### Printing to XML

In [ ]:
print_json_to_xml('/resources/data/Shopify/Nov17.json','/resources/data/Shopify/Nov24.xml')

In [54]:
print_pandas_to_xml(finalShopifySubDf,"Dec1_shopify.xml")

XML file does NOT exist!


In [52]:
def print_pandas_to_xml(dataDf, xml_file):
    editedDataDf = dataDf.replace({'"': '\''}, regex=True)
    try:
        os.remove(xml_file)
    except OSError:
        print("XML file does NOT exist!")
    w = open(xml_file, 'w')
    w.write("<?xml version=\"1.0\" encoding=\"ISO-8859-1\"?>\n")
    w.write("<member>\n")
    for i,row in editedDataDf.iterrows():
        w.write("<heading ")
        w.write("author = \"" + row["author"] + "\" ")
        w.write("time = \"" + datetime.datetime.fromtimestamp(int(row["created_utc"])).strftime('%Y-%m-%d %H:%M:%S') + "\" ")
        w.write("label = \"" + row["id"] + "\" ")
        w.write("title = \"" + row["body"][0:30] + "\" ")
        w.write("tag = \"" + "reddit" + "\"> ")
        w.write(row["full_body"])
        w.write("</heading>\n")
    w.write("</member>")
    w.close()

In [61]:
def print_json_to_xml(json_in,xml_out):
    xml_out = '/resources/data/Shopify/temp.xml'
    json_in = '/resources/data/Shopify/Nov17.json'
    dataDf = read_json_as_pandas('/resources/data/Shopify/Nov17.json')
    print_pandas_to_xml(dataDf,xml_out)